Parte 1: Generación del conjunto de datos

In [ ]:
import pandas as pd
import numpy as np
import unicodedata
import re
import os
import csv
import sklearn.metrics as metrics

In [ ]:
!pip install transformers

In [ ]:
tsv_file = open("lcp_single_test.tsv")
read_tsv = csv.reader(tsv_file, delimiter="\t")

In [ ]:
label = 1
count = 0
data_raw = pd.DataFrame(columns = range(5))
for row in read_tsv:
  if label == 1:
    data_raw.columns = row
    label = 0
  else:
    data_raw.loc[count] = [row[0], row[1], row[2], row[3], row[4]]
    count += 1


In [ ]:
data_raw.head(10)

,id,corpus,sentence,token,complexity
0,3K8CQCU3KE19US5SN890DFPK3SANWR,bible,"But he, beckoning to them with his hand to be ...",hand,0.0
1,3Q2T3FD0ON86LCI41NJYV3PN0BW3MV,bible,"If I forget you, Jerusalem, let my right hand ...",hand,0.19736842105263158
2,3ULIZ0H1VA5C32JJMKOTQ8Z4GUS51B,bible,"the ten sons of Haman the son of Hammedatha, t...",hand,0.2
3,3BFF0DJK8XCEIOT30ZLBPPSRMZQTSD,bible,Let your hand be lifted up above your adversar...,hand,0.2678571428571429
4,3QREJ3J433XSBS8QMHAICCR0BQ1LKR,bible,"Abimelech chased him, and he fled before him, ...",entrance,0.0
5,30F94FBDNRKF12C1PDZGZ0IRI9XTB1,bible,"for as he thinks about the cost, so he is.",cost,0.25
6,3JHB4BPSFK9JLT5CAPRP3PCEA2CQ9R,bible,"When his armor bearer saw that Saul was dead, ...",armor,0.02777777777777779
7,3Y40HMYLL1I1EIURUEH8TTVLKQ9XUM,bible,"Therefore put on the whole armor of God, that ...",armor,0.2
8,30Y6N4AHYPWV0KXTZKLRUB59XBPRDA,bible,"Hezekiah listened to them, and showed them all...",armor,0.2222222222222222
9,38O9DZ0A62NFBR1TA7YCNXN94XN26F,bible,"That first slaughter, which Jonathan and his a...",armor,0.3194444444444444


In [ ]:
data = pd.DataFrame(columns=['text', 'complexity'])
count = 0
for i in range(0, data_raw.shape[0]):
  data.loc[count] = [data_raw['sentence'][count].lower() + ' [SEP] ' + data_raw['token'][count].lower(), \
                    data_raw['complexity'][count]]
  count += 1


In [ ]:
data['text'][0]

'but he, beckoning to them with his hand to be silent, declared to them how the lord had brought him out of the prison. [SEP] hand'

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

In [ ]:
# Preprocess, tokenize and vectorize texts and labels
def vectorize_text(s, max_length):

    # Unicode normalization
    #s = s.strip().lower()
    s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

    # Remove undesired characters
    #s = re.sub(r'([.¿?()\'",;:$€])', r' \1 ', s)
    #s = re.sub(r"[^a-zA-Záéíóú.,!?;:()$€]+", r" ", s)

    # Text to tensor
    input_ids = tokenizer.encode(
      s,
      add_special_tokens=True,
      max_length=max_length,
      padding='longest', 
      truncation=True,
      return_tensors='np'
    )
    return input_ids[0]

In [ ]:
data['text_vec'] = data.apply(lambda r: vectorize_text(r['text'], 512), axis=1)

In [ ]:
data

,text,complexity,text_vec
0,"but he, beckoning to them with his hand to be ...",0.0,"[101, 2021, 2002, 1010, 10272, 13369, 2000, 20..."
1,"if i forget you, jerusalem, let my right hand ...",0.19736842105263158,"[101, 2065, 1045, 5293, 2017, 1010, 6744, 1010..."
2,"the ten sons of haman the son of hammedatha, t...",0.2,"[101, 1996, 2702, 4124, 1997, 10654, 2319, 199..."
3,let your hand be lifted up above your adversar...,0.2678571428571429,"[101, 2292, 2115, 2192, 2022, 4196, 2039, 2682..."
4,"abimelech chased him, and he fled before him, ...",0.0,"[101, 11113, 14428, 2571, 2818, 13303, 2032, 1..."
...,...,...,...
882,"the report by mr philippe busquin, on behalf o...",0.5694444444444444,"[101, 1996, 3189, 2011, 2720, 11169, 3902, 125..."
883,section v - court of auditors (sec(2002) 405 -...,0.5357142857142857,"[101, 2930, 1058, 1011, 2457, 1997, 20964, 201..."
884,"- mr president, on 1 june, on the basis of inf...",0.546875,"[101, 1011, 2720, 2343, 1010, 2006, 1015, 2238..."
885,i would like to extend a warm welcome to this ...,0.5714285714285714,"[101, 1045, 2052, 2066, 2000, 7949, 1037, 4010..."


In [ ]:
# Separar los conjuntos de datos
data = data.sample(frac=1)
train_portion = 0.9
split_point = int(train_portion*len(data))
train_data, test_data =  data[:split_point], data[split_point:] 
print(len(train_data), 'train, ', len(test_data), 'test')

798 train,  89 test


In [ ]:
#Create dataset en pytorch
from torch.utils.data import Dataset, DataLoader
import torch

class MyDataset(Dataset):
    def __init__(self, dataframe):
        self.len = len(dataframe)
        self.data = dataframe
        
    def __getitem__(self, index):
        input_ids = torch.tensor(self.data.text_vec.iloc[index]).cpu()
        attention_mask = torch.ones([input_ids.size(0)]).cpu()
        targets = self.data.complexity.iloc[index]
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': targets
         }
            
    def __len__(self):
        return self.len

In [ ]:
train_set, test_set = MyDataset(train_data), MyDataset(test_data)

In [ ]:
test_set.data

,text,complexity,text_vec
709,"the author, herbert reul, has indicated that h...",0.2361111111111111,"[101, 1996, 3166, 1010, 7253, 2128, 5313, 1010..."
212,"strangers, the terrible of the nations, have c...",0.6666666666666666,"[101, 12358, 1010, 1996, 6659, 1997, 1996, 374..."
288,"however, this study does not support the conce...",0.20588235294117646,"[101, 2174, 1010, 2023, 2817, 2515, 2025, 2490..."
300,their phenotypic features are very similar to ...,0.33333333333333337,"[101, 2037, 6887, 16515, 23186, 2838, 2024, 22..."
105,so yahweh blessed the latter end of job more t...,0.2,"[101, 2061, 8038, 2232, 8545, 2232, 10190, 199..."
...,...,...,...
322,thin layers of articular cartilage line the bo...,0.15625,"[101, 4857, 9014, 1997, 2396, 21412, 11122, 11..."
63,"even though many false witnesses came forward,...",0.2941176470588235,"[101, 2130, 2295, 2116, 6270, 9390, 2234, 2830..."
551,the films and books coming out of iran are imp...,0.171875,"[101, 1996, 3152, 1998, 2808, 2746, 2041, 1997..."
777,"however, who can blame it for behaving in such...",0.27380952380952384,"[101, 2174, 1010, 2040, 2064, 7499, 2009, 2005..."


In [ ]:
train_set.data

,text,complexity,text_vec
742,not only has the vicious circle of violence an...,0.2361111111111111,"[101, 2025, 2069, 2038, 1996, 13925, 4418, 199..."
531,"under basal conditions, the protein resides in...",0.575,"[101, 2104, 15191, 3785, 1010, 1996, 5250, 116..."
411,reversal learning in rodents can serve as an i...,0.26136363636363635,"[101, 23163, 4083, 1999, 28156, 2064, 3710, 20..."
265,we find that dmrt7 protein is expressed only i...,0.3173076923076923,"[101, 2057, 2424, 2008, 1040, 2213, 5339, 2581..."
181,her filthiness was in her skirts; she didn't r...,0.3214285714285714,"[101, 2014, 10882, 24658, 9961, 2001, 1999, 20..."
...,...,...,...
378,both directed and random mutagenesis approache...,0.203125,"[101, 2119, 2856, 1998, 6721, 14163, 26702, 52..."
345,as the ltm7 es cell line that we used in our r...,0.17857142857142855,"[101, 2004, 1996, 8318, 2213, 2581, 9686, 3526..."
689,application of the schengen acquis - applicati...,0.32352941176470584,"[101, 4646, 1997, 1996, 8040, 10222, 6914, 935..."
384,future direct comparison of the two mouse line...,0.27941176470588236,"[101, 2925, 3622, 7831, 1997, 1996, 2048, 8000..."


In [ ]:
len(test_set)

89

In [ ]:
len(train_set)
#len(test_set)

798

In [ ]:
from transformers import BertForSequenceClassification, BertConfig
config = BertConfig.from_pretrained('bert-large-uncased')
config.problem_type='regression'
config.num_labels=1
model = BertForSequenceClassification.from_pretrained('bert-large-uncased', config=config)


Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

In [ ]:
from transformers import Trainer, TrainingArguments

def collate_batch(batch):
    """ Optimize memory by setting all vectors in batch to a length equal to max
        length found
    """
    
    def pad_sequence(in_tensor, max_size):
        """ Fill tensor with zeros up to max_size
        """
        out_tensor = np.zeros(max_size)
        out_tensor[:in_tensor.size(0)] = in_tensor.numpy()
        return out_tensor
    
    batch_inputs = []
    batch_attention_masks = []
    batch_targets = []

    max_size = max([ex['input_ids'].size(0) for ex in batch])
    for item in batch:
        batch_inputs.append(pad_sequence(item['input_ids'], max_size))
        batch_attention_masks.append(pad_sequence(item['attention_mask'], max_size))
        batch_targets.append(float(item['labels']))
    return {
        "input_ids": torch.tensor(batch_inputs, dtype=torch.long),
        "attention_mask": torch.tensor(batch_attention_masks, dtype=torch.long),
        "labels": torch.tensor(batch_targets, dtype=torch.float)
    }

class MyTrainer(Trainer):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)

  def get_train_dataloader(self):
    return DataLoader(
        self.train_dataset,
        collate_fn=collate_batch
    )

  def get_eval_dataloader(self, eval_dataset):
    return DataLoader(
        self.eval_dataset,
        collate_fn=collate_batch
    )


In [ ]:
training_args = TrainingArguments(
    output_dir='output',
    evaluation_strategy='steps',
    eval_steps=100,
    num_train_epochs=5,
    remove_unused_columns=False,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
)

trainer = MyTrainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 798
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3990
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


Step,Training Loss,Validation Loss
100,No log,0.017061
200,No log,0.021632
300,No log,0.019760
400,No log,0.022261
500,0.086900,0.025626
600,0.086900,0.025671
700,0.086900,0.018705
800,0.086900,0.016690
900,0.086900,0.016743
1000,0.068600,0.016690


***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
Saving model checkpoint to output/checkpoint-500
Configuration saved in output/checkpoint-500/config.json
Model weights saved in output/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
Saving model checkpoint to output/checkpoint-1000
Configuration saved in output/checkpoint-1000/config.json
Model weights saved in output/checkpoint-10

TrainOutput(global_step=3990, training_loss=0.06653252938636263, metrics={'train_runtime': 2113.7815, 'train_samples_per_second': 1.888, 'train_steps_per_second': 1.888, 'total_flos': 319194514337310.0, 'train_loss': 0.06653252938636263, 'epoch': 5.0})